In [ ]:
### Run this cell before continuing.
library(tidyverse)
library(repr)
library(tidymodels)
library(cowplot)
options(repr.matrix.max.rows = 6)

## Loading Data

In [ ]:
players <- read_csv("data/players.csv")
sessions <- read_csv("data/sessions.csv")

In [ ]:
players
sessions

## (1) Data Description:
### Dataset Overview

| Dataset | Description | # Observations | # Variables | Key Variable |
|----------|--------------|----------------|--------------|---------------|
| players.csv | Player demographic and skill information | 196 | 7 | hashedEmail |
| sessions.csv | Each recorded game session per player | 1535 | 5 | hashedEmail |


### players.csv — Variable Summary

| Variable | Type | Meaning |
|-----------|------|---------|
| experience | character | Player's self-reported gaming experience level (e.g., Amateur, Pro, Veteran) |
| subscribe | logical | Whether the player subscribed to the game newsletter |
| hashedEmail | character | Anonymized unique player identifier |
| played_hours | numeric | Total number of hours the player has played on the server |
| name | character | Player’s in-game name or alias |
| gender | character | Player’s gender (Male, Female, Other, or Prefer not to say) |
| Age | numeric | Player’s age in years (some missing values) |

---

### sessions.csv — Variable Summary

| Variable | Type | Meaning |
|-----------|------|---------|
| hashedEmail | character | Unique player ID (foreign key linking to players.csv) |
| start_time | character | Start time of a game session |
| end_time | character | End time of a game session |
| original_start_time | numeric | Original start time as a UNIX timestamp |
| original_end_time | numeric| Original end time as a UNIX timestamp |

---

### Potential Data Issues and Observations

| Category | Description | Possible Impact |
|-----------|--------------|----------------|
| Missing values | `Age` contains missing data; `gender` includes “Prefer not to say” | Could reduce sample size or introduce bias |
| Outliers | Some players have 0 or unusually high `played_hours` | May skew averages or affect regression results |
| Data type inconsistencies | `start_time` and `end_time` are stored as character, not datetime | Need conversion with `lubridate` for time-based calculations |
| Duplicates | Players may appear multiple times in `sessions.csv` | Must aggregate sessions per player |
| Sampling bias | Data comes from a voluntary Minecraft research server | May not represent the general player population |
| Ethical considerations | All identifiers are anonymized (`hashedEmail`) | Satisfies data privacy and ethics requirements |

---


### How the Data Were Collected

The data were collected from a Minecraft research server operated by the UBC Computer Science department.  
Player information (`players.csv`) was obtained through voluntary registration forms, including demographics, experience, and newsletter subscription.  
Session data (`sessions.csv`) were automatically logged by the server, recording start and end times for each play session.  
All players were anonymized using hashed identifiers (`hashedEmail`) to ensure privacy and comply with research ethics.


### Summary Statistics for Numeric Variables in `players.csv` （See Part 3)

| Variable     | Min | Mean  | Median | Max   | SD   |
|---------------|------|--------|---------|--------|--------|
| Age           | 9.00 | 21.14 | 19.00 | 58.00 | 7.39 |
| played_hours  | 0.00 | 5.85  | 0.10  | 223.10 | 28.36 |


## (2) Questions

### Broad Question
What player characteristics and behaviours are most predictive of subscribing to a game-related newsletter, and how do these features differ between various player types?

### Specific Predictive Question
Which combination of player features — such as **age**, **gender**, **experience level**, **total playtime**, and **average session duration (avg_session_min)** — best predicts whether a player will subscribe to the newsletter, and how much does each factor contribute to improving the prediction accuracy?

### How the Data Help
The `players.csv` dataset includes each player's demographic information (`Age`, `gender`), self-reported experience level (`experience`), total playtime (`played_hours`), and subscription status (`subscribe`).  
The `sessions.csv` dataset records detailed play sessions, from which we can calculate each player's **average session duration (`avg_session_min`)**.  
These combined data allow us to test how demographic and behavioural features, both individually and together, relate to subscription behaviour and which combinations produce the most accurate predictions.

### Data Wrangling Plan
- Use both `players.csv` and `sessions.csv` datasets.  
- Keep `subscribe` as the variable we aim to predict.  
- Use `Age`, `gender`, `experience`, `played_hours`, and `avg_session_min` as predictors, testing different feature combinations.  
- Handle missing values (e.g., missing `Age` or incomplete session data) and ensure all categorical variables are properly encoded.  
- Convert `experience` to a factor variable for classification.  
- Calculate `avg_session_min` from session data as:
  \[
  avg\_session\_min = \frac{end\_time - start\_time}{\text{number of sessions}}
  \]
- Split the final dataset into **training (80%)** and **testing (20%)** sets, and compare model performance to identify which feature combination best predicts subscription.


## (3) Exploratory Data Analysis and Visualization:
### 1. Data Loading and Tidying
The `players.csv` dataset was successfully loaded into R and converted into a tidy format, where each row represents one player and each column represents a variable.  
Only minimal wrangling was performed to ensure consistent data types (e.g., converting `experience` and `gender` into factors).

In [ ]:
## Minimal Data Wrangling to Create Tidy Datasets

players_tidy <- players |>
  mutate(
    experience    = as.factor(experience),
    gender        = as.factor(gender))



sessions_tidy <- sessions |>
  mutate(
    start_time  = dmy_hm(start_time, quiet = TRUE),
    end_time    = dmy_hm(end_time, quiet = TRUE),
    duration_min = as.numeric(difftime(end_time, start_time, units = "mins")))

session_summary <- sessions_tidy %>%
  group_by(hashedEmail) %>%
  summarise(
    avg_session_min = mean(duration_min, na.rm = TRUE),
    session_count = n(),
    .groups = "drop"
  )

#merge with players.csv
players_extended <- left_join(players, session_summary, by = "hashedEmail")

players_extended


In [ ]:
# calculating the means of Age average session duration and played_hours
mean_Age <- mean(players$Age, na.rm = TRUE)
mean_played_hours <- mean(players$played_hours, na.rm = TRUE)

mean_values <- data.frame(
  Variable = c("Age", "played_hours""),
  Mean = round(c(mean_Age, mean_played_hours), 2)
)

mean_values

In [ ]:
# exploratory visualizations
# Distribution of player age
ggplot(players, aes(x = Age)) +
  geom_histogram() +
  labs(title = "Distribution of Player Ages",
       x = "Age (years)",
       y = "Number of Players")

# Distribution of total playtime
ggplot(players, aes(x = played_hours)) +
  geom_histogram() +
  labs(title = "Distribution of Total Playtime",
       x = "Total Playtime (hours)",
       y = "Number of Players")

# Playtime by Experience
ggplot(players, aes(x = experience, y = played_hours)) +
  geom_point(alpha = 0.3) +
  labs(title = "Playtime by Experience Level",
       x = "Experience Level",
       y = "Played Hours") 

# Bar chart: Subscription rate by Experience
players |>
  group_by(experience) |>
  summarise(sub_rate = mean(subscribe, na.rm = TRUE)) |>
  ggplot(aes(x = experience, y = sub_rate)) +
  geom_col() +
  labs(title = "Newsletter Subscription Rate by Experience Level",
       x = "Experience Level",
       y = "Subscription Rate (0–1)")

# Bar chart: Subscription rate by Age
players |>
  group_by(Age) |>
  summarise(sub_rate = mean(subscribe, na.rm = TRUE)) |>
  ggplot(aes(x = Age, y = sub_rate)) +
  geom_col() +
  labs(title = "Newsletter Subscription Rate by Age",
       x = "Age",
       y = "Subscription Rate (0–1)")

# Bar chart: Subscription rate by gender
players |>
  group_by(gender) |>
  summarise(sub_rate = mean(subscribe, na.rm = TRUE)) |>
  ggplot(aes(x = gender, y = sub_rate)) +
  geom_col() +
  labs(title = "Newsletter Subscription Rate by Gender",
       x = "Gender",
       y = "Subscription Rate (0–1)")

# Scatter plot: Subscription rate by average session duration
players_extended |>
  group_by(avg_session_min) |>
  summarise(sub_rate = mean(subscribe, na.rm = TRUE)) |>
  ggplot(aes(x = avg_session_min, y = sub_rate)) +
  geom_point(alpha = 0.3) +
  labs(title = "Newsletter Subscription Rate by average session duration",
       x = "average session duration(min)",
       y = "Subscription Rate (0–1)")

# Scatter plot: Subscription rate by Playtime
players |>
  group_by(played_hours) |>
  summarise(sub_rate = mean(subscribe, na.rm = TRUE)) |>
  ggplot(aes(x = played_hours, y = sub_rate)) +
  geom_point(alpha = 0.3) +
  labs(title = "Newsletter Subscription Rate by Total Playtime",
       x = "Total Playtime (hours)",
       y = "Subscription Rate (0–1)")

# Scatter plot: Age vs average session duration, colored by Subscription
ggplot(players_extended, aes(x = Age, y = avg_session_min, color = subscribe)) +
  geom_point(size = 2, alpha = 0.8) +
  labs(title = "Age vs. average session duration (colored by Subscription)",
       x = "Age (years)",
       y = "average session duration(min)",
       color = "Subscribed?")


# Scatter plot: Age vs Playtime, colored by Subscription
ggplot(players, aes(x = Age, y = played_hours, color = subscribe)) +
  geom_point(size = 2, alpha = 0.8) +
  labs(title = "Age vs. Total Playtime (colored by Subscription)",
       x = "Age (years)",
       y = "Total Playtime (hours)",
       color = "Subscribed?")


### Insights from the Visualizations

- **Age and playtime** appear to be the most informative predictors of subscription.  
- **Experience** and **gender** show weaker or inconsistent effects.  
- Some groups (e.g., “Veteran” or “Other” gender) may have **small sample sizes**, leading to unstable proportions.  





## (4) Methods and Plan

### Proposed Method
I plan to use a **K-Nearest Neighbours (KNN)** classification model。
The goal is to predict whether a player subscribes to the newsletter (`subscribe`) based on demographic and behavioural features such as `Age`, `gender`, `experience`, `played_hours`, and `avg_session_min`.

### Why this method is appropriate
KNN makes predictions based on the similarity between observations.  
It does not assume any specific functional relationship between predictors and the response.  
This makes it suitable for our data, where the relationship between player behaviour and subscription likelihood may be nonlinear.

### Assumptions
- The data is representative of the population and independent across players.  
- Predictors are scaled or normalized, so no variable dominates distance calculations.  
- The dataset is relatively free of extreme outliers, which can distort distance measures.

### Potential Limitations
- Model performance depends on the choice of **k** (number of neighbours). Too small a k may overfit, while too large may oversmooth patterns.  
- Sensitive to irrelevant or highly correlated features.  
- KNN can be slower for larger datasets, since distances must be computed for each prediction.  
- It does not directly show which features are most influential, only overall accuracy.

### Model Comparison and Selection
I will tune the hyperparameter **k** using **cross-validation**, comparing accuracy scores for different values of k (e.g., k = 1–10).  
The best-performing k will be chosen based on the lowest cross-validation error or highest accuracy on validation folds.

### Data Processing Plan
Before modelling:
1. Select relevant variables: `subscribe` (response), `Age`, `gender`, `experience`, `played_hours`, and `avg_session_min` (predictors).  
2. Handle missing values and convert categorical variables (e.g., `gender`, `experience`) to factors.  
3. Normalize quantitative predictors (`Age`, `played_hours`, `avg_session_min`) to ensure fair distance computation.  
4. Split the dataset into **training (80%)** and **testing (20%)** sets.  
5. Use **5-fold cross-validation** on the training data to select the best k.  
6. Evaluate model accuracy on the test data.




## (5) GitHub Repository

https://github.com/Lissiu/dsci-100-2025w1-group-008-13